In [1]:
from tracker import Tracker
import pickle as pkl
import cv2
import os
import numpy as np
import xml.etree.ElementTree as ET

In [2]:
def process_video(video_path, annotations_path, detections_path, output_path, tracking_output):
    with open(detections_path, 'rb') as f:
        data = pkl.load(f)
    
    cap = cv2.VideoCapture(video_path)
    width = int(cap.get(3))
    height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, height))

    tracker = Tracker()
    #juntar les dos funcions següents i passar test split idx a tot arreu on hi diu 535
    tracker.parse_bboxes_from_xml(annotations_path)
    tracker.read_gt_ids(annotations_path)
    frame_idx = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if frame_idx < 535:
            frame_idx += 1
            continue
        
        detections = data[frame_idx-535]
        tracker.update(detections, tracker.gt_detections[frame_idx-535])
        frame = tracker.draw_tracks(frame)
        out.write(frame)
        
        frame_idx += 1

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    
    tracker.tracking_data["num_tracker_ids"] = tracker.next_id
    
    with open(tracking_output, 'wb') as f:
        pkl.dump(tracker.tracking_data, f)


In [3]:
process_video('/Users/abriil/Uni/master/C6/Project1/data/AICity_data/train/S03/c010/vdo.avi', '/Users/abriil/Uni/master/C6/Project1/data/ai_challenge_s03_c010-full_annotation.xml', 'preds_pred_off-shelf.pkl', 'output_video.avi', 'tracking_outputs_bo.pkl')

In [4]:
with open('/Users/abriil/Uni/master/C6/Project1/mcv-c6-2025-team2/week2/tracking/tracking_outputs_bo.pkl', 'rb') as f:
        t_out = pkl.load(f)
        t_out['num_gt_ids'] = t_out['num_gt_ids'][0]
        #t_out['gt_ids'] = t_out['gt_ids'][535:]

In [5]:
from TrackEval.hota import HOTA

In [6]:
h = HOTA()

In [8]:
h.eval_sequence(t_out)

{'HOTA': array([0.27518306, 0.27457751, 0.27428683, 0.27353913, 0.27283019,
        0.27159804, 0.26807204, 0.25354993, 0.24241644, 0.24048252,
        0.23927573, 0.23829021, 0.23609038, 0.23355395, 0.23201283,
        0.23082694, 0.22672459, 0.17422648, 0.12698694]),
 'DetA': array([0.67885859, 0.67622382, 0.67480852, 0.67087846, 0.66586683,
        0.65840549, 0.63046031, 0.51005982, 0.40224197, 0.38884721,
        0.38234855, 0.37591042, 0.35642065, 0.33298133, 0.31980419,
        0.31275409, 0.29796681, 0.18009519, 0.08324232]),
 'AssA': array([0.11154859, 0.11149091, 0.11148832, 0.11153086, 0.11178859,
        0.11203657, 0.11398437, 0.12603927, 0.14609547, 0.14872639,
        0.14974   , 0.15105254, 0.15638451, 0.16381533, 0.1683216 ,
        0.17036092, 0.172516  , 0.16854901, 0.19371978]),
 'DetRe': array([0.68657903, 0.6849893 , 0.68413329, 0.6817487 , 0.67869153,
        0.67410578, 0.65655763, 0.57352492, 0.48706817, 0.47538979,
        0.46964231, 0.46389483, 0.44616325, 0

In [9]:
from TrackEval.identity import Identity

In [10]:
iden = Identity()


Identity Config:
THRESHOLD            : 0.5                           
PRINT_CONFIG         : True                          


In [11]:
iden.eval_sequence(t_out)

{'IDF1': 0.19632697155203457,
 'IDR': 0.16667685723020484,
 'IDP': 0.23880858519491896,
 'IDTP': 2726,
 'IDFN': 13629,
 'IDFP': 8689}